In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import joblib
import os

# Cargar datos
print("Cargando datos...")
df = pd.read_csv('../data/data.csv')
print(f"Dataset shape: {df.shape}")

# Definir las categorías de toxicidad detallada
toxicity_categories = [
    'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist', 
    'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual', 
    'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 
    'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 
    'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation', 
    'physical_disability', 'psychiatric_or_mental_illness', 'transgender', 
    'white', 'sexual_explicit'
]

# Verificar qué categorías existen en el dataset
available_categories = [cat for cat in toxicity_categories if cat in df.columns]
print(f"Categorías disponibles: {len(available_categories)}/{len(toxicity_categories)}")

# Filtrar solo comentarios tóxicos (target > 0.5) con datos completos
print("Filtrando comentarios tóxicos...")
toxic_df = df[df['target'] > 0.5].copy()
print(f"Comentarios tóxicos iniciales: {len(toxic_df)}")

# Limpiar datos - eliminar filas con valores NaN en las categorías
for category in available_categories:
    toxic_df = toxic_df.dropna(subset=[category])

print(f"Después de limpiar NaN: {len(toxic_df)}")

# Convertir a binario (umbral de 0.5)
for category in available_categories:
    toxic_df[category] = (toxic_df[category] > 0.5).astype(int)

# Filtrar categorías que tienen al menos algunos ejemplos positivos
valid_categories = []
for category in available_categories:
    positive_count = toxic_df[category].sum()
    total_count = len(toxic_df)
    if positive_count >= 5 and positive_count < total_count - 5:  # Al menos 5 de cada clase
        valid_categories.append(category)
        print(f"{category}: {positive_count}/{total_count} ({positive_count/total_count*100:.1f}%)")
    else:
        print(f"❌ {category}: {positive_count}/{total_count} - insuficientes datos")

print(f"\nCategorías válidas para entrenamiento: {len(valid_categories)}")

if len(valid_categories) == 0:
    raise ValueError("No hay categorías con suficientes datos para entrenar")

# Preparar datos solo con categorías válidas
X = toxic_df['comment_text']
y = toxic_df[valid_categories]

print(f"Datos finales: X={X.shape}, y={y.shape}")

# Dividir datos
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y.iloc[:, 0]
    )
except ValueError:
    # Si falla la estratificación, usar división simple
    print("Usando división aleatoria (sin estratificación)")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# Vectorizar texto
print("Vectorizando texto...")
vectorizer_detailed = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2
)

X_train_vec = vectorizer_detailed.fit_transform(X_train)
X_test_vec = vectorizer_detailed.transform(X_test)
print(f"Vectorized shape: {X_train_vec.shape}")

# Entrenar modelo multi-etiqueta
print("Entrenando modelo multi-etiqueta...")
detailed_classifier = MultiOutputClassifier(
    LogisticRegression(random_state=42, max_iter=1000)
)

detailed_classifier.fit(X_train_vec, y_train)
print("✅ Entrenamiento completado")

# Evaluación simplificada
print("\nEvaluando modelo...")
y_pred = detailed_classifier.predict(X_test_vec)

print("Métricas por categoría:")
for i, category in enumerate(valid_categories):
    y_true = y_test.iloc[:, i]
    y_pred_cat = y_pred[:, i]
    
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    precision = precision_score(y_true, y_pred_cat, zero_division=0)
    recall = recall_score(y_true, y_pred_cat, zero_division=0)
    f1 = f1_score(y_true, y_pred_cat, zero_division=0)
    support = y_true.sum()
    
    print(f"{category}: P={precision:.3f} R={recall:.3f} F1={f1:.3f} Support={support}")

# Crear directorio artifacts_detail si no existe
os.makedirs('../artifacts_detail', exist_ok=True)

# Guardar modelos
print("\nGuardando modelos...")
joblib.dump(detailed_classifier, '../artifacts_detail/detailed_model.pkl')
joblib.dump(vectorizer_detailed, '../artifacts_detail/vectorizer_detailed.pkl')
joblib.dump(valid_categories, '../artifacts_detail/toxicity_categories.pkl')

print("✅ Modelos guardados exitosamente!")
print(f"- detailed_model.pkl")
print(f"- vectorizer_detailed.pkl")
print(f"- toxicity_categories.pkl")

print("🎉 Proceso completado!")

Cargando datos...
Dataset shape: (90902, 45)
Categorías disponibles: 29/29
Filtrando comentarios tóxicos...
Comentarios tóxicos iniciales: 45451
Después de limpiar NaN: 12246
obscene: 1479/12246 (12.1%)
identity_attack: 2566/12246 (21.0%)
insult: 10209/12246 (83.4%)
threat: 362/12246 (3.0%)
asian: 93/12246 (0.8%)
atheist: 34/12246 (0.3%)
bisexual: 6/12246 (0.0%)
black: 977/12246 (8.0%)
buddhist: 10/12246 (0.1%)
christian: 536/12246 (4.4%)
female: 1531/12246 (12.5%)
heterosexual: 27/12246 (0.2%)
hindu: 10/12246 (0.1%)
homosexual_gay_or_lesbian: 583/12246 (4.8%)
❌ intellectual_or_learning_disability: 0/12246 - insuficientes datos
jewish: 252/12246 (2.1%)
latino: 50/12246 (0.4%)
male: 1396/12246 (11.4%)
muslim: 959/12246 (7.8%)
❌ other_disability: 0/12246 - insuficientes datos
❌ other_gender: 0/12246 - insuficientes datos
other_race_or_ethnicity: 8/12246 (0.1%)
other_religion: 5/12246 (0.0%)
❌ other_sexual_orientation: 1/12246 - insuficientes datos
❌ physical_disability: 0/12246 - insufic